In [1]:
!pip install simpy
!pip install tabulate

In [2]:
# STEP 1: Import libraries
import numpy as np
from matplotlib import pyplot as plt
import random
import simpy
import pandas as pd
from scipy import stats
from tabulate import tabulate

In [3]:
# GARMENT FACTORY - INTRODUCTORY EXAMPLE

import simpy
import numpy as np
import random

materials_capacity = 100
SIM_TIME = 100

class Factory:
  def __init__(self, env):
    self.materials = simpy.Container(env, capacity = materials_capacity, init = materials_capacity)
    self.finished_shirts = simpy.Container(env, capacity = materials_capacity, init = 0)

def shirt_mkr(env, garment_factory):
  while True:
    yield garment_factory.materials.get(1)
    shirt_making_time = np.random.triangular(0, 3, 10)
    yield env.timeout(shirt_making_time)
    yield garment_factory.finished_shirts.put(1)

print('Garment Factory')
env = simpy.Environment()

garment_factory = Factory(env)

shirt_maker_gen = env.process(shirt_mkr(env, garment_factory))

env.run(until=SIM_TIME)

print('Our factory made {} shirts'.format(garment_factory.finished_shirts.level))
print('We still have materials to make {} more shirts'.format(garment_factory.materials.level))
print('End of simulation')


Garment Factory
Our factory made 24 shirts
We still have materials to make 75 more shirts
End of simulation


In [4]:
# GARMENT FACTORY - FULL EXAMPLE

# --------------------------------------
# STEP 2: DEFINE SIMULATION PARAMETERS
# --------------------------------------

RANDOM_SEED = 42
SIM_TIME = 60 * 24 * 7  # running simulation for 7 days, base time is minutes

# REPORTING VARIALBES - 
# define variables to store simulation results
num_orders_completed = 0
order_in_lst = []
order_out_lst = []
serger_wait = []
finishing_wait = []
assembling_wait = []
body_wait = []
sleeve_wait = []
collar_wait = []
pass_ctrl = []
fail_ctrl = []
ctrl_wait= []
qty_making = []
order_sizes = []

body_level = []
sleeves_level = []
collar_level = []

# Change capacity as the factory 'grows'
min_order = 100
max_order = 1000

# -----------------------------
# RESOURCES
# -----------------------------

num_serging_mc = 7
num_body_mkrs = 6
num_sleeve_mkrs = 3
num_collar_mkrs = 3
num_assembler = 4
num_finisher = 3
num_inspector = 3

#We can declare values of distribution as global var or inside function
serge_time_min = 3
serge_time_max = 6



# -----------------------------
# CONTAINERS (STORE MATERIALS)
# -----------------------------

bodies_capacity = 2500       
collars_capacity = 5000
sleeves_capacity = 2500

min_stock = 70  


# -----------------------------
# STEP 3: CREATE FACTORY CLASS - DEFINE PROCESSES
# -----------------------------

class Factory:
  def __init__(self, env):
    # stock control and dispatch
    self.stock_ctrl = env.process(self.stock_ctrl(env))
    self.bodies = simpy.Container(env, capacity = bodies_capacity, init = 0)
    self.collars = simpy.Container(env, capacity = collars_capacity, init = 0)
    self.sleeves = simpy.Container(env, capacity = sleeves_capacity, init = 0)
    self.dispatch_ctrl = env.process(self.dispatch_ctrl(env))
    self.current_order_qty = 0
    
    # serging
    self.serger = simpy.Resource(env, num_serging_mc)
    self.post_serge_bodies = simpy.Container(env, capacity = bodies_capacity, init = 0)
    self.post_serge_collars = simpy.Container(env, capacity = collars_capacity, init = 0)
    self.post_serge_sleeves = simpy.Container(env, capacity = sleeves_capacity, init = 0)

    # body
    self.body_mkr = simpy.Resource(env, num_body_mkrs)
    self.post_body_mkr = simpy.Container(env, capacity = bodies_capacity, init = 0)

    # sleeves
    self.sleeve_mkr = simpy.Resource(env, num_sleeve_mkrs)
    self.post_sleeve_mkr = simpy.Container(env, capacity = sleeves_capacity, init = 0)

    # collar
    self.collar_mkr = simpy.Resource(env, num_collar_mkrs)
    self.post_collar_mkr = simpy.Container(env, capacity = collars_capacity, init = 0)

    # assembling
    self.assembler = simpy.Resource(env, num_assembler)
    self.post_assembling = simpy.Container(env, capacity = max_order, init = 0)

    # finishing
    self.finisher = simpy.Resource(env, num_finisher)
    self.post_finishing = simpy.Container(env, capacity = max_order, init = 0)

    # quality control
    self.qc = simpy.Resource(env, num_inspector)
    self.dispatch = simpy.Container(env, capacity = 2 * max_order, init = 0)

  def stock_ctrl(self, env):
    global qty_making 
    global order_in_lst
    global body_level
    global sleeves_level
    global collar_level
    yield env.timeout(0)
    while True:
      if self.bodies.level <= min_stock or self.collars.level <= min_stock or self.sleeves.level <= min_stock:
        print("Stock below critical level at time {0}. Processing next order.".format(env.now))
        yield env.timeout(random.randint(60, 120)) #  It takes X time to process the new order 
        order_qty = random.randint(min_order, max_order) 
        order_sizes.append(order_qty)
        if self.current_order_qty == 0:
          self.current_order_qty = order_qty
          #print("initial order quty is {}".format(self.current_order_qty))
        #print('New order qty is {}'.format(order_qty))  
        matl_qty = int(order_qty * 1.15) # material quantity will account for items that fail quality control 
        qty_making.append(matl_qty)
        #order_in = env.now
        order_in_lst.append(env.now)
        print("Accepting new order of {0} and sourcing material for {1} shirts at time {2}.".format(order_qty, matl_qty, env.now))
        yield self.bodies.put(matl_qty *2)
        yield self.sleeves.put(matl_qty *2)
        yield self.collars.put(matl_qty *4)
        print("Piece stock levels: bodies - {0}, collars - {1}, sleeves - {2}".format(self.bodies.level, self.collars.level, self.sleeves.level))
        body_level.append(self.bodies.level)
        sleeves_level.append(self.sleeves.level)
        collar_level.append(self.collars.level)
        yield env.timeout(60 * 2)
      else:
        yield env.timeout(60)
 
  def dispatch_ctrl(self, env):
    global order_out_lst

    yield env.timeout(0)
    print('checking dispatch at time {}'.format(env.now))
    while True:
      #print('inside dispatch loop')
      if env.now >= 180:
        if self.dispatch.level >= self.current_order_qty:
          order_out_lst.append(env.now)
          print("Calling order company to inform them their order is ready at time {}".format(env.now))
          wait_time = int(np.random.uniform(60 * 4, 60 * 10))
          yield env.timeout(wait_time)
          print("Order company has picked up their order at time {}".format(env.now))
          global num_orders_completed 
          #print('access global var')
          num_orders_completed += 1
          print("We have now completed {} orders".format(num_orders_completed))
          print("before pickup, we had {} shirts in dispatch".format(self.dispatch.level))
          self.dispatch.get(self.current_order_qty)
          print("After pickup, there are {} shirts waiting in dispatch".format(self.dispatch.level))
          self.current_order_qty = order_sizes[num_orders_completed]
          #print("now processing order {0} + 1 of {1}".format(num_orders_completed, self.current_order_qty))
          yield env.timeout(60 * 3)
        else:
          yield env.timeout(10)
      else: 
        yield env.timeout(60)

# -----------------------------
# STEP 4: Use functions to define the processes in the simulation
# -----------------------------

def serger(env, garment_factory):
  global serger_wait
  while True:
    
    yield garment_factory.sleeves.get(2)
    yield garment_factory.bodies.get(2)
    yield garment_factory.collars.get(4)
    serger_in = env.now #if we count time in before resource retrieval, we will end up with really high processing time for later processes
    serging_time = np.random.uniform(serge_time_min, serge_time_max) #the process takes between [min, mix] time
    yield env.timeout(serging_time)
    yield garment_factory.post_serge_sleeves.put(2)
    yield garment_factory.post_serge_bodies.put(2)
    yield garment_factory.post_serge_collars.put(4)
    serger_out = env.now
    serger_wait.append(serger_out - serger_in)
    

def body_mkr(env, garment_factory):
  global body_wait
  while True:
    
    yield garment_factory.post_serge_bodies.get(2)
    body_in = env.now
    # time it takes to print or add design to a shirt. For some it's 0
    print_design_time = np.random.triangular(0, 3, 10) #the process is optional, so it takes 0 time minimum, most likely takes 3 mins and 10 mins at most
    #This process is done by a machine therefore there is less variations.
    yield env.timeout(print_design_time)
    body_make_time = max(np.random.normal(2, .5), 1) # process never < 1 mins.
    #this process is a normal distribution because it heavily relies on the speed of the worker
    yield env.timeout(body_make_time)
    # Two body peices now combined to one 
    yield garment_factory.post_body_mkr.put(1)
    body_out = env.now
    body_wait.append(body_out - body_in)
    #print("Post body maker level: {0}".format(garment_factory.post_body_mkr.level))

def sleeve_mkr(env, garment_factory):
  global sleeve_wait
  while True:
    
    yield garment_factory.post_serge_sleeves.get(2)
    sleeve_in = env.now
    sleeve_make_time = max(np.random.normal(3, 1), 1)
    #this process is a normal distribution because it heavily relies on the speed of the worker
    yield env.timeout(sleeve_make_time)
    yield garment_factory.post_sleeve_mkr.put(2) #make 2 sleeves at a time
    sleeve_out = env.now
    sleeve_wait.append(sleeve_out-sleeve_in)
    #print("Post sleeve maker level: {0}".format(garment_factory.post_sleeve_mkr.level))

def collar_mkr(env, garment_factory):
  global collar_wait
  while True:
    
    yield garment_factory.post_serge_collars.get(4) #4 pieces makes 1 collar
    collar_in = env.now
    collar_make_time = max(np.random.normal(4, 1,1)[0], 1)
    #this process is a normal distribution because it heavily relies on the speed of the worker
    yield env.timeout(collar_make_time)
    yield garment_factory.post_collar_mkr.put(1)
    collar_out = env.now
    collar_wait.append(collar_out-collar_in)
    #print("Post collar maker level: {0}".format(garment_factory.post_collar_mkr.level))

def assembling(env, garment_factory):
  global assembling_wait
  while True:
    
    yield garment_factory.post_collar_mkr.get(1)
    yield garment_factory.post_body_mkr.get(1)
    yield garment_factory.post_sleeve_mkr.get(2)
    assem_in = env.now
    assembling_time = max(np.random.normal(6, 2,1)[0], 0) # Control so we don't get negative values
    #this process is a normal distribution because it heavily relies on the speed of the worker
    yield env.timeout(assembling_time)
    yield garment_factory.post_assembling.put(1)
    assem_out = env.now
    assembling_wait.append(assem_out-assem_in)
    #print("Post assembling level: {0}".format(garment_factory.post_assembling.level))

def finishing(env, garment_factory): 
  global finishing_wait
  while True:
    
    # We can do washing and binding in batches of 10
    yield garment_factory.post_assembling.get(10)
    fin_in = env.now
    
    binding_time = np.random.uniform(5, 10)
    #this process is a normal distribution because it heavily relies on the speed of the worker
    washing_time = np.random.triangular(20, 24, 30)
    #this process is done in batch by machine, depending on the type of wash, it can take different amount of time. 
    #Most popular process takes 24 mins
    yield env.timeout(binding_time)
    yield env.timeout(washing_time)
    yield garment_factory.post_finishing.put(10)
    fin_out = env.now
    
    finishing_wait.append(fin_out - fin_in)
    #print("There are now {0} shirts in post finishing".format(garment_factory.post_finishing.level))

def qc(env, garment_factory):
  #get finished product run through bern 
  #put passed product to dispatch
  global ctrl_wait
  while True:
    yield garment_factory.post_finishing.get(10)
    ctrl_in = env.now
    check_time = np.random.triangular(2, 3, 5) 
    #This process is highly standardized so it has low variabtion, mostlikely takes 3 mins/ batch
    result_lst = np.random.binomial(size=10, n=1, p=0.875)
    #generate random quality check decision, we use binomial with n=1 which is bern distribution
    #we hard coded the estimated probability in this case.
    #however, if we were to simulate the system to improve overall product quality, this would not be an appropriate distribution
    num_pass = np.count_nonzero(result_lst == 1)
    pass_ctrl.append(num_pass)
    num_fail = np.count_nonzero(result_lst == 0)
    fail_ctrl.append(num_fail)
    yield env.timeout(check_time)
    yield garment_factory.dispatch.put(num_pass)
    ctrl_out= env.now
    ctrl_wait.append(ctrl_out-ctrl_in)


# -----------------------------
# GENERATORS - GENERATE PROCESSES FOR EACH RESOURCE
# -----------------------------

def serger_gen(env, garment_factory): 
  for i in range(num_serging_mc):
    env.process(serger(env, garment_factory))
    yield env.timeout(0)

def body_mkr_gen(env, garment_factory):
  for i in range(num_body_mkrs):
    env.process(body_mkr(env, garment_factory))
    yield env.timeout(0)

def sleeve_mkr_gen(env, garment_factory):
  for i in range(num_sleeve_mkrs):
    env.process(sleeve_mkr(env, garment_factory))
    yield env.timeout(0)

def collar_mkr_gen(env, garment_factory):
  for i in range(num_collar_mkrs):
    env.process(collar_mkr(env, garment_factory))
    yield env.timeout(0)
  
def assembler_mkr_gen(env, guitar_factory):
  for i in range(num_assembler):
    env.process(assembling(env, garment_factory))
    yield env.timeout(0)

def finisher_mkr_gen(env, garment_factory):
  for i in range(num_finisher):
    env.process(finishing(env, garment_factory))
    yield env.timeout(0)

def quality_ctrl_gen(env, garment_factory):
  for i in range(num_inspector):
    env.process(qc(env, garment_factory))
    yield env.timeout(0)

# -----------------------------
# RUN SIMULATION (STEPS 5-7)
# -----------------------------

# STEP 5: Set up the SimPy environment
print('Garment Factory')
random.seed(RANDOM_SEED)
env = simpy.Environment()

garment_factory = Factory(env)

# STEP 6: Generate the processes for the simulation
# generate serger, bodies, sleeves, collars, assembling, binding, washing by calling above generators
sergers_gen = env.process(serger_gen(env, garment_factory))
body_mkrs_gen = env.process(body_mkr_gen(env, garment_factory))
sleeve_mkrs_gen = env.process(sleeve_mkr_gen(env, garment_factory))
collar_mkrs_gen = env.process(collar_mkr_gen(env, garment_factory))
assembler_gen = env.process(assembler_mkr_gen(env, garment_factory))
finisher_gen = env.process(finisher_mkr_gen(env, garment_factory))
qc_gen = env.process(quality_ctrl_gen(env, garment_factory))

# STEP 7: Run the simulation and check its output
env.run(until=SIM_TIME) #SIM_TIME

print('End of simulation')

Garment Factory
Stock below critical level at time 0. Processing next order.
checking dispatch at time 0
Accepting new order of 214 and sourcing material for 246 shirts at time 100.
Piece stock levels: bodies - 478, collars - 984, sleeves - 478
Stock below critical level at time 280. Processing next order.
Accepting new order of 859 and sourcing material for 987 shirts at time 341.
Piece stock levels: bodies - 1960, collars - 3948, sleeves - 1960
Calling order company to inform them their order is ready at time 530
Order company has picked up their order at time 809
We have now completed 1 orders
before pickup, we had 381 shirts in dispatch
After pickup, there are 167 shirts waiting in dispatch
Stock below critical level at time 1001. Processing next order.
Accepting new order of 350 and sourcing material for 402 shirts at time 1078.
Piece stock levels: bodies - 790, collars - 1608, sleeves - 790
Stock below critical level at time 1318. Processing next order.
Accepting new order of 242

In [5]:
# Material level _ Stock Control Chart 
stock_level_df = pd.DataFrame(list(zip(body_level, sleeves_level, collar_level)),
               columns =['Body_Pieces_In_Stock', 'Sleeve_Pieces_In_Stock','Collar_Pieces_In_Stock'])
stock_level_df['Sleeves_to_body_ratio'] = stock_level_df['Sleeve_Pieces_In_Stock']/stock_level_df['Body_Pieces_In_Stock'] #should be 1
stock_level_df['Collar_to_body_ratio'] = stock_level_df['Collar_Pieces_In_Stock']/stock_level_df['Body_Pieces_In_Stock'] #should be 2

stock_level_df

,Body_Pieces_In_Stock,Sleeve_Pieces_In_Stock,Collar_Pieces_In_Stock,Sleeves_to_body_ratio,Collar_to_body_ratio
0,478,478,984,1.0,2.058577
1,1960,1960,3948,1.0,2.014286
2,790,790,1608,1.0,2.035443
3,542,542,1112,1.0,2.051661
4,454,454,936,1.0,2.061674
5,1958,1958,3944,1.0,2.014300
6,1498,1498,3024,1.0,2.018692
7,1604,1604,3236,1.0,2.017456
8,288,288,604,1.0,2.097222
9,434,434,896,1.0,2.064516


In [6]:
# Order Size and Order Time
stat_df = pd.DataFrame(list(zip(order_sizes, qty_making, order_in_lst, order_out_lst)),
               columns =['Order_Size', 'Quantity_Making','Time_Received_Order', 'Time_Finished_Order'])

stat_df['Order_Time'] = stat_df['Time_Finished_Order'] - stat_df['Time_Received_Order']
stat_df['Avg_Item_Time'] = stat_df['Order_Time']/stat_df['Quantity_Making']
print('Variance of production time', stat_df.var()['Avg_Item_Time']) #adjust lead time between when to receive new order helps
stat_df



Variance of production time 29.359702271352717


,Order_Size,Quantity_Making,Time_Received_Order,Time_Finished_Order,Order_Time,Avg_Item_Time
0,214,246,100,530,430,1.747967
1,859,987,341,1989,1648,1.669706
2,350,402,1078,2594,1516,3.771144
3,242,278,1392,3327,1935,6.960432
4,204,234,1679,3852,2173,9.286325
5,858,986,1962,4785,2823,2.863083
6,658,756,2739,5948,3209,4.244709
7,704,809,3644,7147,3503,4.330037
8,132,151,4751,7666,2915,19.304636
9,195,224,4932,8211,3279,14.638393


In [7]:
# Total Shirts Made
print("Total number of shirts made", np.sum(stat_df['Quantity_Making']))

Total number of shirts made 6284


In [8]:
# Quality Control Statistics
print('Statistics of the Quality Control Process')
print('Total pass rate of quality check', np.sum(pass_ctrl)/ (np.sum(pass_ctrl)+np.sum(fail_ctrl)))
print('Total fail rate of quality check', 1-(np.sum(pass_ctrl)/ (np.sum(pass_ctrl)+np.sum(fail_ctrl))))

Statistics of the Quality Control Process
Total pass rate of quality check 0.8754147812971342
Total fail rate of quality check 0.12458521870286576


In [9]:
# Descriptive Statistics 
print('Descriptive statistics of the System')
stats_lst = [serger_wait, body_wait, sleeve_wait, collar_wait, assembling_wait, finishing_wait, ctrl_wait]
stats_names = ['Serging 1 body, 2 sleeves and 1 collar', 'Sewing 1 body', 'Sewing 2 sleeves', ' Sewing 1 collar', "Assembling 1 shirt", 'Finishing 10 shirts',\
               'Check quality of 10 shirts']
table_head = ['Num_Observation', 'Min_Max', 'Mean', 'Variance']
for i in range(len(stats_lst)):
  print()
  print()
  print('Descriptive Statistic for the process of', stats_names[i])
  i_stats = []
  for u in range(4):
    i_stats.append(stats.describe(stats_lst[i])[u])
  i_table = list([table_head]+[i_stats])
  print(tabulate(i_table))

Descriptive statistics of the System


Descriptive Statistic for the process of Serging 1 body, 2 sleeves and 1 collar
---------------  ---------------------------------------  -----------------  -----------------
Num_Observation  Min_Max                                  Mean               Variance
8757             (3.000999151651513, 14.238115802912034)  7.163729132368175  6.958177718282864
---------------  ---------------------------------------  -----------------  -----------------


Descriptive Statistic for the process of Sewing 1 body
---------------  ----------------------------------------  -----------------  -----------------
Num_Observation  Min_Max                                   Mean               Variance
8755             (1.2200335052848459, 12.487503091750114)  6.331907622539313  4.619443050810294
---------------  ----------------------------------------  -----------------  -----------------


Descriptive Statistic for the process of Sewing 2 sleeves
---------------  -